In [1]:
import pandas as pd
from pathlib import Path
import os

In [2]:
proj_dir="/master/nplatt/sch_hae_scan"
results_dir=f"{proj_dir}/results"
os.chdir(results_dir)

In [3]:
Path(f"{results_dir}/fig_plotting_info").mkdir(parents=True, exist_ok=True)
os.chdir(f"{results_dir}/fig_plotting_info")

In [7]:
spacer_size=2_500_000

In [36]:
len_df.iloc[:8]

,contig,length,x_offset,x_start,x_end,mid_tick,color
0,NC_067195.1,89168492,0.0,0.0,89168492.0,44584246.0,na
1,NC_067196.1,93306550,91668492.0,91668492.0,184975042.0,92487521.0,na
2,NC_067197.1,47367863,187475042.0,187475042.0,234842905.0,117421452.5,na
3,NC_067198.1,48328128,237342905.0,237342905.0,285671033.0,142835516.5,na
4,NC_067199.1,46771736,288171033.0,288171033.0,334942769.0,167471384.5,na
5,NC_067200.1,22148653,337442769.0,337442769.0,359591422.0,179795711.0,na
6,NC_067201.1,25509538,362091422.0,362091422.0,387600960.0,193800480.0,na
7,NC_067202.1,19481189,390100960.0,390100960.0,409582149.0,204791074.5,na


In [46]:
len_df = pd.read_csv(f"{proj_dir}/data/GCF_000699445.3_UoM_Shae.V3_genomic.fna.fai", sep="\t", header=None)
len_df.columns=["contig", "length", "offset", "linebases", "linewidth"]


# Take out the row with index 0
sex_chr = len_df.iloc[0]

# Drop the row with index 0 from the original DataFrame
len_df = len_df.drop(0)

# Add the row back at position 7
len_df = pd.concat([len_df.iloc[:7], pd.DataFrame(sex_chr).T, len_df.iloc[7:]]).reset_index(drop=True)


len_df['cumsum'] = len_df['length'].shift().cumsum().fillna(0)

contig_offset = len_df.set_index('contig')['cumsum'].to_dict()
spacers = {contig: i * spacer_size for i, contig in enumerate(contig_offset.keys())}


len_df[["x_offset", "x_start", "x_end", "mid_tick", "color"]]="na"

for contig in contig_offset.keys():
    contig_offset[contig] += spacers[contig]

for index, row in len_df.iterrows():
    contig = row["contig"]
    length = row["length"]
    
    offset = contig_offset[contig]
    start  = offset
    end    = start + length
    tick   = (start + end)/2
    
    len_df.loc[len_df["contig"]==contig, "x_offset"] = offset
    len_df.loc[len_df["contig"]==contig, "x_start"] = start
    len_df.loc[len_df["contig"]==contig, "x_end"] = end
    len_df.loc[len_df["contig"]==contig, "mid_tick"] = tick
    
len_df.drop(["offset", "linebases", "linewidth", "cumsum"], axis=1, inplace=True)

len_df.head(10)

,contig,length,x_offset,x_start,x_end,mid_tick,color
0,NC_067196.1,93306550,0.0,0.0,93306550.0,46653275.0,na
1,NC_067197.1,47367863,95806550.0,95806550.0,143174413.0,119490481.5,na
2,NC_067198.1,48328128,145674413.0,145674413.0,194002541.0,169838477.0,na
3,NC_067199.1,46771736,196502541.0,196502541.0,243274277.0,219888409.0,na
4,NC_067200.1,22148653,245774277.0,245774277.0,267922930.0,256848603.5,na
5,NC_067201.1,25509538,270422930.0,270422930.0,295932468.0,283177699.0,na
6,NC_067202.1,19481189,298432468.0,298432468.0,317913657.0,308173062.5,na
7,NC_067195.1,89168492,320413657.0,320413657.0,409582149.0,364997903.0,na
8,NW_026137020.1,17113,412082149.0,412082149.0,412099262.0,412090705.5,na
9,NW_026137021.1,57163,414599262.0,414599262.0,414656425.0,414627843.5,na


In [47]:
contigs = len_df['contig']
colors= ["#078d70", "#7bade3", "#3e1a78"]*50000
color_map = dict(zip(contigs, colors))

colors=[]

for index, row in len_df.iterrows():
    colors.append(color_map[row['contig']])
    
len_df["color"] = colors
len_df.head(10)



,contig,length,x_offset,x_start,x_end,mid_tick,color
0,NC_067196.1,93306550,0.0,0.0,93306550.0,46653275.0,#078d70
1,NC_067197.1,47367863,95806550.0,95806550.0,143174413.0,119490481.5,#7bade3
2,NC_067198.1,48328128,145674413.0,145674413.0,194002541.0,169838477.0,#3e1a78
3,NC_067199.1,46771736,196502541.0,196502541.0,243274277.0,219888409.0,#078d70
4,NC_067200.1,22148653,245774277.0,245774277.0,267922930.0,256848603.5,#7bade3
5,NC_067201.1,25509538,270422930.0,270422930.0,295932468.0,283177699.0,#3e1a78
6,NC_067202.1,19481189,298432468.0,298432468.0,317913657.0,308173062.5,#078d70
7,NC_067195.1,89168492,320413657.0,320413657.0,409582149.0,364997903.0,#7bade3
8,NW_026137020.1,17113,412082149.0,412082149.0,412099262.0,412090705.5,#3e1a78
9,NW_026137021.1,57163,414599262.0,414599262.0,414656425.0,414627843.5,#078d70


In [48]:
len_df.to_csv("plotting_info.csv", sep=",", index=False, header=True, na_rep="Na")